In [175]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress
from functools import reduce
# Study data files
crime_path = "NYPD_Complaint_Data_Historic.csv"
covid_path = "NYC/COVID-19_Daily_Counts_of_Cases__Hospitalizations__and_Deaths.csv"

# Read the mouse data and the study results
crime_data = pd.read_csv(crime_path)
covid_data = pd.read_csv(covid_path)

In [154]:
crime_data.drop(crime_data.columns.difference(["CMPLNT_FR_DT","OFNS_DESC","BORO_NM"]),1,inplace=True)
crime_data=crime_data.rename(columns={"CMPLNT_FR_DT": "Report_Date", "OFNS_DESC": "Offense","BORO_NM":"Borough"}) 
crime_data["Report_Date"]=pd.to_datetime(crime_data["Report_Date"])
crime_data.head()


,Report_Date,Offense,Borough
0,2020-02-01,RAPE,BROOKLYN
1,2020-02-01,RAPE,BROOKLYN
2,2020-02-01,PETIT LARCENY,STATEN ISLAND
3,2020-02-01,THEFT-FRAUD,MANHATTAN
4,2020-02-01,ROBBERY,BRONX


In [155]:
crime_df=crime_data.loc[(crime_data["Report_Date"]<="2020-12-31") & (crime_data["Report_Date"]>="2020-03-01")]
crime_df

,Report_Date,Offense,Borough
35330,2020-03-01,CRIMINAL TRESPASS,BROOKLYN
35331,2020-03-01,CRIMINAL MISCHIEF & RELATED OF,MANHATTAN
35332,2020-03-01,CRIMINAL MISCHIEF & RELATED OF,BROOKLYN
35333,2020-03-01,ASSAULT 3 & RELATED OFFENSES,QUEENS
35334,2020-03-01,GRAND LARCENY,BRONX
...,...,...,...
366454,2020-12-31,DANGEROUS WEAPONS,BROOKLYN
366455,2020-12-31,PETIT LARCENY,MANHATTAN
366456,2020-12-31,GRAND LARCENY,MANHATTAN
366457,2020-12-31,DANGEROUS WEAPONS,BROOKLYN


In [156]:
bronx=crime_df.loc[(crime_df["Borough"]=="BRONX")]
bronx
bronx_df = pd.DataFrame(bronx.groupby(["Borough","Report_Date"])["Offense"].count().reset_index())
bronx_df

,Borough,Report_Date,Offense
0,BRONX,2020-03-01,269
1,BRONX,2020-03-02,267
2,BRONX,2020-03-03,264
3,BRONX,2020-03-04,280
4,BRONX,2020-03-05,275
...,...,...,...
301,BRONX,2020-12-27,195
302,BRONX,2020-12-28,217
303,BRONX,2020-12-29,182
304,BRONX,2020-12-30,196


In [157]:
queens=crime_df.loc[(crime_df["Borough"]=="QUEENS")]
queens_df = pd.DataFrame(queens.groupby(["Borough","Report_Date"])["Offense"].count().reset_index())
queens_df

,Borough,Report_Date,Offense
0,QUEENS,2020-03-01,283
1,QUEENS,2020-03-02,258
2,QUEENS,2020-03-03,240
3,QUEENS,2020-03-04,241
4,QUEENS,2020-03-05,231
...,...,...,...
301,QUEENS,2020-12-27,172
302,QUEENS,2020-12-28,200
303,QUEENS,2020-12-29,172
304,QUEENS,2020-12-30,156


In [158]:
brooklyn=crime_df.loc[(crime_df["Borough"]=="BROOKLYN")]
brooklyn_df = pd.DataFrame(brooklyn.groupby(["Borough","Report_Date"])["Offense"].count().reset_index())
brooklyn_df

,Borough,Report_Date,Offense
0,BROOKLYN,2020-03-01,361
1,BROOKLYN,2020-03-02,366
2,BROOKLYN,2020-03-03,402
3,BROOKLYN,2020-03-04,393
4,BROOKLYN,2020-03-05,363
...,...,...,...
301,BROOKLYN,2020-12-27,242
302,BROOKLYN,2020-12-28,249
303,BROOKLYN,2020-12-29,270
304,BROOKLYN,2020-12-30,252


In [159]:

manhattan=crime_df.loc[(crime_df["Borough"]=="MANHATTAN")]
manhattan_df = pd.DataFrame(manhattan.groupby(["Borough","Report_Date"])["Offense"].count().reset_index())
manhattan_df

,Borough,Report_Date,Offense
0,MANHATTAN,2020-03-01,290
1,MANHATTAN,2020-03-02,298
2,MANHATTAN,2020-03-03,339
3,MANHATTAN,2020-03-04,329
4,MANHATTAN,2020-03-05,339
...,...,...,...
301,MANHATTAN,2020-12-27,170
302,MANHATTAN,2020-12-28,220
303,MANHATTAN,2020-12-29,230
304,MANHATTAN,2020-12-30,224


In [160]:

si=crime_df.loc[(crime_df["Borough"]=="STATEN ISLAND")]
si_df = pd.DataFrame(si.groupby(["Borough","Report_Date"])["Offense"].count().reset_index())
si_df

,Borough,Report_Date,Offense
0,STATEN ISLAND,2020-03-01,55
1,STATEN ISLAND,2020-03-02,54
2,STATEN ISLAND,2020-03-03,49
3,STATEN ISLAND,2020-03-04,57
4,STATEN ISLAND,2020-03-05,51
...,...,...,...
301,STATEN ISLAND,2020-12-27,42
302,STATEN ISLAND,2020-12-28,44
303,STATEN ISLAND,2020-12-29,32
304,STATEN ISLAND,2020-12-30,46


In [189]:
covid_data.drop(covid_data.columns.difference(["DATE_OF_INTEREST","CASE_COUNT","BX_CASE_COUNT","BK_CASE_COUNT","MN_CASE_COUNT","QN_CASE_COUNT","SI_CASE_COUNT"]),1,inplace=True) 
covid_data["DATE_OF_INTEREST"]=pd.to_datetime(covid_data["DATE_OF_INTEREST"])
covid_data=covid_data.rename(columns={"DATE_OF_INTEREST": "Report_Date"}) 
covid_data

,Report_Date,CASE_COUNT,BX_CASE_COUNT,BK_CASE_COUNT,MN_CASE_COUNT,QN_CASE_COUNT,SI_CASE_COUNT
0,2020-02-29,1,0,0,1,0,0
1,2020-03-01,0,0,0,0,0,0
2,2020-03-02,0,0,0,0,0,0
3,2020-03-03,1,0,0,0,1,0
4,2020-03-04,5,0,1,2,2,0
...,...,...,...,...,...,...,...
509,2021-07-22,883,125,302,199,178,79
510,2021-07-23,812,107,297,160,192,55
511,2021-07-24,478,52,163,84,129,50
512,2021-07-25,555,60,195,90,146,64


In [190]:
#covid_data.loc[(covid_data["DATE_OF_INTEREST"]<="2020-12-31") & (covid_data["DATE_OF_INTEREST"]>="2020-03-01")]
covid_data=covid_data.loc[(covid_data["Report_Date"]<="2020-12-31") & (covid_data["Report_Date"]>="2020-03-01")]
covid_data


,Report_Date,CASE_COUNT,BX_CASE_COUNT,BK_CASE_COUNT,MN_CASE_COUNT,QN_CASE_COUNT,SI_CASE_COUNT
1,2020-03-01,0,0,0,0,0,0
2,2020-03-02,0,0,0,0,0,0
3,2020-03-03,1,0,0,0,1,0
4,2020-03-04,5,0,1,2,2,0
5,2020-03-05,3,0,3,0,0,0
...,...,...,...,...,...,...,...
302,2020-12-27,2886,479,913,314,868,312
303,2020-12-28,5385,1018,1600,674,1531,562
304,2020-12-29,5201,991,1413,609,1602,585
305,2020-12-30,4975,966,1393,658,1492,465


False

In [207]:
combined_data=pd.merge(covid_data,bronx_df, on="Report_Date")
combined_data1=pd.merge(combined_data,brooklyn_df, on="Report_Date")
combined_data2=pd.merge(combined_data1,queens_df, on="Report_Date")
combined_data3=pd.merge(combined_data2,manhattan_df, on="Report_Date")
combined_data4=pd.merge(combined_data3,si_df, on="Report_Date")
combined_final=combined_data4
combined_final.head()
combined_final=combined_final.rename(columns={"Report_Date": "REPORT_DATE", "Borough_x": "BX_BOROUGH","Offense_x":"BX_OFFENSE","Borough_y":"MN_Borough","Offense_y":"MN_OFFENSE",}) 


    
# frames=[covid_data,bronx_df,queens_df,manhattan_df,brooklyn_df,si_df]
# frame=reduce(lambda left,right: pd.merge(left,right, on="Report_Date"), frames)

,Report_Date,CASE_COUNT,BX_CASE_COUNT,BK_CASE_COUNT,MN_CASE_COUNT,QN_CASE_COUNT,SI_CASE_COUNT,Borough_x,Offense_x,Borough_y,Offense_y,Borough_x,Offense_x,Borough_y,Offense_y,Borough,Offense
0,2020-03-01,0,0,0,0,0,0,BRONX,269,BROOKLYN,361,QUEENS,283,MANHATTAN,290,STATEN ISLAND,55
1,2020-03-02,0,0,0,0,0,0,BRONX,267,BROOKLYN,366,QUEENS,258,MANHATTAN,298,STATEN ISLAND,54
2,2020-03-03,1,0,0,0,1,0,BRONX,264,BROOKLYN,402,QUEENS,240,MANHATTAN,339,STATEN ISLAND,49
3,2020-03-04,5,0,1,2,2,0,BRONX,280,BROOKLYN,393,QUEENS,241,MANHATTAN,329,STATEN ISLAND,57
4,2020-03-05,3,0,3,0,0,0,BRONX,275,BROOKLYN,363,QUEENS,231,MANHATTAN,339,STATEN ISLAND,51


In [205]:


# covid_df = pd.DataFrame(covid_data.groupby("DATE_OF_INTEREST")["BX_CASE_COUNT"].count().reset_index())
# covid_df
# covid_df=covid_df.set_index("DATE_OF_INTEREST")
# covid_df
#covid,= plt.plot(covid_data["DATE_OF_INTEREST"], covid_data["BX_CASE_COUNT"], color="blue", label="Bronx Cases" )
#plt.legend(handles=[covid], loc="best")

# Show the chart
#plt.show()
# Start by getting the last (greatest) timepoint for each mouse
# sorted_data = crime_data.sort_values(["Report_Date", "Borough"], ascending=True)
# sorted_data.head(50)

# # Select final volume of each mouse 
# final_volume = sorted_data.loc[sorted_data["Timepoint"] == 45]
# final_volume.head()